In [ ]:
import json
import random
import pandas as pd
import jsonlines
import warnings
import plotly.graph_objects as go

warnings.filterwarnings("ignore", category=pd.errors.DtypeWarning)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.colheader_justify', 'left') 
pd.set_option('display.max_colwidth', None)  
pd.set_option('display.width', None)  

In [ ]:
file_path = '/home/carolus/Documents/school/green_ia/data/10_data/10_train_02.jsonl'
#file_path_bis = '/home/carolus/Documents/school/green_ia/data/17_data/17_openfoodfacts_02.jsonl'
num_samples = 500

In [ ]:
def count_objects_in_jsonl(file_path):
    count = 0
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            count += 1
    return count
print(f"nombre objets {file_path}: {count_objects_in_jsonl(file_path)}")


In [ ]:
def sample_jsonl_file(file_path, num_samples):
    sample_lines = []

    with open(file_path, 'r') as file:
        line_count = 0
        for line in file:
            line_count += 1
            if len(sample_lines) < num_samples:
                sample_lines.append(line)
            else:
                idx = random.randint(0, line_count - 1)
                if idx < num_samples:
                    sample_lines[idx] = line

    data_list = [json.loads(line) for line in sample_lines]
    df = pd.DataFrame(data_list)
    return df
df = sample_jsonl_file(file_path, num_samples)

In [ ]:
#occurrences = df['food_group'].value_counts()
#print(occurrences)

In [ ]:
df.tail(1)

In [ ]:
df.tail(100)

In [ ]:
col_nan = 'additives'
nb_empty = (df[col_nan].isna()).sum() 
#nb_empty = (df[col_nan] == 'empty').sum() 
nb_total = len(df)
pourcentage_empty = (nb_empty / nb_total) * 100
print(f"percetange empty in {col_nan}: {pourcentage_empty:.2f}%")

In [ ]:
#def load_jsonl_to_dataframe(file_path):
#    data = []
#    with open(file_path, 'r') as file:
#        for line in file:
#            data.append(json.loads(line))
#    df = pd.DataFrame(data)
#    return df
#
#def create_treemap_for_column(df, column_name):
#    if column_name not in df.columns:
#        print(f"warning, column: '{column_name}'does not exists")
#        return
#    data = df[column_name].value_counts().reset_index()
#    data.columns = [column_name, 'count']
#    fig = go.Figure(go.Treemap(
#        labels=data[column_name],
#        parents=[""] * len(data),
#        values=data['count'],
#        textinfo='label+value',  
#    ))
#    fig.update_layout(
#        title=f'treemap column: {column_name}',
#        paper_bgcolor='white',  
#        plot_bgcolor='white',   
#        font_color='black'      
#    )
#    fig.show()

In [ ]:
#colonnes_a_travailler = ['stores']  
#df = load_jsonl_to_dataframe(file_path)
#for column in colonnes_a_travailler:
#    create_treemap_for_column(df, column)

In [ ]:
#nb_empty = df['ecoscore_score'].isna().sum()
#nb_total = len(df)
#pourcentage_empty = (nb_empty / nb_total) * 100
#print(f"percetange empty in ecoscore_score before NaN reduction : {pourcentage_empty:.2f}%")

In [ ]:
#nb_empty = df['countries'].isna().sum()
#nb_total = len(df)
#pourcentage_empty = (nb_empty / nb_total) * 100
#print(f"percetange empty in countries before NaN reduction : {pourcentage_empty:.2f}%")

In [ ]:
#nb_empty = df['groups'].isna().sum()
#nb_total = len(df)
#pourcentage_empty = (nb_empty / nb_total) * 100
#print(f"percetange empty in groups before NaN reduction : {pourcentage_empty:.2f}%")

In [ ]:
#nb_empty = df['text_data'].isna().sum()
#nb_total = len(df)
#pourcentage_empty = (nb_empty / nb_total) * 100
#print(f"percetange empty in text_data before NaN reduction : {pourcentage_empty:.2f}%")

In [ ]:
#nb_empty = df['labels_note'].isna().sum()
#nb_total = len(df)
#pourcentage_empty = (nb_empty / nb_total) * 100
#print(f"percetange empty in labels_note before NaN reduction : {pourcentage_empty:.2f}%")

In [ ]:
#nb_empty = df['categories'].isna().sum()
#nb_total = len(df)
#pourcentage_empty = (nb_empty / nb_total) * 100
#print(f"percetange empty in categories before NaN reduction : {pourcentage_empty:.2f}%")

In [ ]:
#file_path_csv = '/home/carolus/Documents/school/green_ia/scripts/validation_predictions.csv'
#df_csv = pd.read_csv(file_path_csv)

In [ ]:
#df_csv.head(60)

In [ ]:
#df_csv['difference'] = df_csv['true'] - df_csv['predictions']
#max_diff = df_csv['difference'].max()
#min_diff = df_csv['difference'].min()
#mean_diff = df_csv['difference'].mean()  
#median_diff = df_csv['difference'].median()  
#total_rows = df_csv.shape[0]
#df_csv['difference'] = df_csv['true'] - df_csv['predictions']
#count_greater_than_2 = (df_csv['difference'].abs() > 2).sum()
#percentage = (count_greater_than_2 / total_rows) * 100

#print(f"Écart maximum: {max_diff}")
#print(f"Écart minimum: {min_diff}")
#print(f"Écart moyen: {mean_diff}")
#print(f"Écart médian: {median_diff}")
#print(f"poucentage lignes écart supérieur à 2% : {percentage}%")